# Generation: Stuffing Documents

In [1]:
%load_ext dotenv
%dotenv

In [4]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

In [5]:
vectorstore = Chroma(persist_directory = "./intro-to-ds_lectures",
                     embedding_function = OpenAIEmbeddings(model="text-embedding-ada-002"))

In [6]:
len(vectorstore.get()["documents"])

21

In [7]:
retriever = vectorstore.as_retriever(search_type = "mmr", search_kwargs = {"k": 3, "lambda_mult": 0.7})

In [9]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is taken from in the format:
Resources: *Lecture Title*
where *Lecture Title* should be substituted with the title of all resource lectures.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [10]:
chat = ChatOpenAI(model_name = "gpt-4",
                  model_kwargs = {"seed": 365},
                  max_tokens = 250)

C:\Users\Asus\.conda\envs\Langchain_env\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [11]:
question = "What Software do data scientists use?"

In [29]:
chain = ({"context":retriever, "question": RunnablePassthrough()} | prompt_template | chat | StrOutputParser())

In [30]:
chain.invoke(question)

'Data scientists often utilize software such as R and Python, two highly popular tools in the field. R and Python are very adaptable and capable of handling a wide variety of business and data-related problems. They are often integrated within various data and data science software platforms. In addition, Hadoop, a software framework designed for the complexity of big data, is also used. It distributes computational tasks on multiple computers. There are also several examples of software specially designed for business intelligence visualizations, including Power BI, SaS, Qlik, and Tableau.\n\nResources: Programming Languages & Software Employed in Data Science - All the Tools You Need'

In [25]:
print("Data scientists often utilize software such as R and Python, two highly adaptable tools which are not only popular but also capable of handling a wide variety of business and data-related issues. They are often integrated within multiple data and data science software platforms and are suitable for mathematical and statistical computations. Another significant software tool is Hadoop, a software framework created to manage the complexity and computational intensity of big data by distributing tasks across multiple computers. Furthermore, software designed for business intelligence visualizations, like Power BI, SaS, Qlik, and especially Tableau, are also used by data scientists.\n\nResources: Programming Languages & Software Employed in Data Science - All the Tools You Need")

Data scientists often utilize software such as R and Python, two highly adaptable tools which are not only popular but also capable of handling a wide variety of business and data-related issues. They are often integrated within multiple data and data science software platforms and are suitable for mathematical and statistical computations. Another significant software tool is Hadoop, a software framework created to manage the complexity and computational intensity of big data by distributing tasks across multiple computers. Furthermore, software designed for business intelligence visualizations, like Power BI, SaS, Qlik, and especially Tableau, are also used by data scientists.

Resources: Programming Languages & Software Employed in Data Science - All the Tools You Need
